# Aggregation and Translation with Cortex AISQL

This notebook demonstrates:
- **AI_AGG**: Aggregate insights across rows
- **AI_SUMMARIZE_AGG**: Aggregate summaries
- **AI_TRANSLATE**: Translate text
- **AI_FILTER**: Semantic filtering and joins


In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from snowflake.snowpark import Session
from IPython.display import display, Markdown, HTML

# Try to get active session (for Snowflake Notebooks)
# Otherwise, connect using ~/.snowflake/connections.toml
try:
    from snowflake.snowpark.context import get_active_session
    session = get_active_session()
    print("✅ Connected using Snowflake Notebooks session")
except:
    import toml
    from pathlib import Path
    toml_path = Path.home() / ".snowflake" / "connections.toml"
    if toml_path.exists():
        connections = toml.load(toml_path)
        connection_name = "vinodshiv"
        if connection_name in connections:
            conn_params = connections[connection_name]
            session = Session.builder.configs(conn_params).create()
            print(f"✅ Connected using connection profile: {connection_name}")
        else:
            raise Exception(f"❌ Connection '{connection_name}' not found")
    else:
        raise Exception(f"❌ Connection file not found at {toml_path}")

session.sql("USE DATABASE AISQL_DB").collect()
session.sql("USE SCHEMA AISQL_SCHEMA").collect()
session.sql("USE WAREHOUSE AISQL_WH").collect()


✅ Connected using connection profile: vinodshiv


[Row(status='Statement executed successfully.')]

## 1. AI_AGG: Aggregate Insights

Aggregate insights across multiple tickets


In [2]:
# Aggregate insights by day
sql = """
SELECT 
    DATE_TRUNC('day', created_at) as date,
    COUNT(*) as ticket_count,
    AI_AGG(content, 'Summarize the main issues reported today') as daily_insights
FROM emails
GROUP BY DATE_TRUNC('day', created_at)
ORDER BY date DESC
LIMIT 7
"""

df_agg = session.sql(sql).to_pandas()
display(Markdown("### Daily Aggregated Insights"))
display(df_agg)


### Daily Aggregated Insights

,DATE,TICKET_COUNT,DAILY_INSIGHTS
0,2025-01-19,3,Two main issues are reported: \n1. Duplicate p...
1,2025-01-18,3,Main issues reported today: \n1. Mobile ticket...
2,2025-01-17,2,Sound quality issues at Stage 3 with constant ...
3,2025-01-16,2,Two main issues were reported: \n1. The sound ...
4,2025-01-15,6,Main issues reported today include: \n1. Mobil...
5,2025-01-14,1,The main issue reported is the need to resolve...
6,2025-01-13,3,Discrepancy in account refund for the canceled...


## 2. AI_TRANSLATE: Multilingual Support

Translate customer emails to multiple languages


In [3]:
# Translate emails to multiple languages
sql = """
SELECT 
    ticket_id,
    SUBSTR(content, 1, 100) as original_english,
    AI_TRANSLATE(content, 'en', 'es') as spanish,
    AI_TRANSLATE(content, 'en', 'fr') as french,
    AI_TRANSLATE(content, 'en', 'de') as german
FROM emails
LIMIT 5
"""

df_translate = session.sql(sql).to_pandas()
display(Markdown("### Multilingual Translations"))

for idx, row in df_translate.iterrows():
    display(Markdown(f"#### Ticket {row['TICKET_ID']}"))
    display(Markdown(f"**English:** {row['ORIGINAL_ENGLISH']}"))
    display(Markdown(f"**Spanish:** {row['SPANISH'][:100] if row['SPANISH'] else 'N/A'}"))
    display(Markdown(f"**French:** {row['FRENCH'][:100] if row['FRENCH'] else 'N/A'}"))
    display(Markdown(f"**German:** {row['GERMAN'][:100] if row['GERMAN'] else 'N/A'}"))
    display(Markdown("---"))


### Multilingual Translations

#### Ticket 1830

**English:** I'd appreciate a refund for the unused Saturday ticket (Order #TR789456) or at least credit toward a

**Spanish:** Agradecería un reembolso por el boleto no utilizado del sábado (Pedido #TR789456) o al menos un créd

**French:** J'apprécierais un remboursement pour le billet non utilisé du samedi (Commande #TR789456) ou au moin

...

**English:** Also, I purchased tickets for the Summer Electronica Festival next month (Order #EF-2023-8867), but 

**Spanish:** Además, compré boletos para el Festival de Electrónica de Verano del próximo mes (Pedido #EF-2023-88

**French:** De plus, j'ai acheté des billets pour le Festival Electronica d'été le mois prochain (Commande #EF-2

**German:** Außerdem habe ich Tickets für das Summer Electronica Festival im nächsten Monat gekauft (Bestellung 

---

## 3. AI_SUMMARIZE_AGG: Aggregate Summaries

Create coherent summaries across multiple rows


In [4]:
# Aggregate summaries by user
sql = """
SELECT 
    user_id,
    COUNT(*) as ticket_count,
    AI_SUMMARIZE_AGG(content) as user_summary
FROM emails
GROUP BY user_id
HAVING COUNT(*) >= 2
ORDER BY ticket_count DESC
LIMIT 10
"""

df_sum_agg = session.sql(sql).to_pandas()
display(Markdown("### User Issue Summaries"))
display(df_sum_agg)


### User Issue Summaries

,USER_ID,TICKET_COUNT,USER_SUMMARY
0,15,2,Attended Summer Vibes Festival ticket holder n...
1,39,2,"Drink prices on the mobile app are outdated, s..."
2,129,2,Two refunds are pending: one for the Electroni...
